## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [30]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
# home = 1 if player A is home, -1 if not, 0 if neutral
def EloRating(Ra, Rb, K, tie, home, homeWeight): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb)
    if home == 1: 
        Ra = Ra-homeWeight
        Rb = Rb+homeWeight
    elif home == -1:
        Ra = Ra+homeWeight
        Rb = Rb-homeWeight
        
    return Ra, Rb

In [31]:
gameFilename = 'data/2019NFLseason.txt'
playoffgameFilename = 'data/2019NFLplayoffs.txt'
teamFilename = 'data/NFLteams.txt'

K = 31
homeWeight = -90
Round1_K = 31
Round1_homeWeight = -90
Round2_K = 31
Round2_homeWeight = -90
Round3_K = 31
Round3_homeWeight = -90

### Load the team names into an array

In [32]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [33]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
numGames = len(games)
playoffgames = pd.read_csv(playoffgameFilename, header = None)
numPlayoffGames = len(playoffgames)

### Create Elo ratings

In [34]:
import numpy as np

eloRatings = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, False, games.loc[i, 3], homeWeight)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], K, False, games.loc[i, 6], homeWeight)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, True, games.loc[i, 3], homeWeight)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

### Sort and print the ranking of teams

In [35]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   200.59707   Kansas_City
   2   179.18488   Baltimore
   3   138.73682   San_Francisco
   4   130.47501   Houston
   5   116.06027   New_England
   6   96.74613   New_Orleans
   7   79.99626   Minnesota
   8   68.82779   Green_Bay
   9   56.92478   Tennessee
  10   56.40594   Seattle
  11   43.45180   Chicago
  12   25.70920   Buffalo
  13   16.95107   Philadelphia
  14   12.41750   Denver
  15   11.66826   Atlanta
  16   -5.61816   Dallas
  17   -23.53021   Carolina
  18   -26.26071   Cincinnati
  19   -29.18370   NY_Jets
  20   -30.30946   LA_Rams
  21   -33.32099   Cleveland
  22   -39.51867   Pittsburgh
  23   -51.22862   Indianapolis
  24   -53.54222   Tampa_Bay
  25   -78.35599   NY_Giants
  26   -100.14120   Arizona
  27   -108.96847   Miami
  28   -109.55536   Detroit
  29   -118.90135   Washington
  30   -123.09568   Jacksonville
  31   -130.80626   Las_Vegas
  32   -171.81573   LA_Chargers



In [36]:
#Brier score for playoffs
def PlayoffsBrier():
    TotBrierScore = 0
    numBrierScore = 0
    for i in range(numPlayoffGames):
        team1ID = playoffgames.loc[i, 2] - 1 
        team1Score = playoffgames.loc[i, 4]
        team2ID = playoffgames.loc[i, 5] - 1 
        team2Score = playoffgames.loc[i, 7]

        if team1Score > team2Score:
            TotBrierScore = TotBrierScore + ((1-Probability(eloRatings[team2ID], eloRatings[team1ID]))**2)
            numBrierScore += 1
            if i < 4:
                team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], Round1_K, False, playoffgames.loc[i, 3], Round1_homeWeight)
            if i < 8:
                team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], Round2_K, False, playoffgames.loc[i, 3], Round2_homeWeight)
            if i < 10:
                team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], Round3_K, False, playoffgames.loc[i, 3], Round3_homeWeight)
                
        elif team2Score > team1Score:
            TotBrierScore = TotBrierScore + ((1-Probability(eloRatings[team1ID], eloRatings[team2ID]))**2)
            numBrierScore += 1
            if i < 4:
                team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], Round1_K, False, playoffgames.loc[i, 6], Round1_homeWeight)
            if i < 8:
                team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], Round2_K, False, playoffgames.loc[i, 6], Round2_homeWeight)
            if i < 10:
                team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], Round3_K, False, playoffgames.loc[i, 6], Round3_homeWeight)
                
        if i < 10:
            eloRatings[team1ID] = team1Rating
            eloRatings[team2ID] = team2Rating
        
    AvgBrierScore = TotBrierScore/numBrierScore
    print(f'Brier Score: {AvgBrierScore:.5f}' + f' K: {K}' + f' homeWeight: {homeWeight}') 

PlayoffsBrier()

Brier Score: 0.21710 K: 31 homeWeight: -90


In [37]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   400.45291   Kansas_City
   2   341.09515   San_Francisco
   3   245.82504   Baltimore
   4   187.94727   New_England
   5   170.49945   New_Orleans
   6   124.56554   Houston
   7   93.20372   Philadelphia
   8   67.95708   Green_Bay
   9   43.45180   Chicago
  10   12.41750   Denver
  11   11.66826   Atlanta
  12   -5.61816   Dallas
  13   -23.53021   Carolina
  14   -26.26071   Cincinnati
  15   -29.18370   NY_Jets
  16   -30.30946   LA_Rams
  17   -33.32099   Cleveland
  18   -39.51867   Pittsburgh
  19   -51.22862   Indianapolis
  20   -53.54222   Tampa_Bay
  21   -75.25362   Buffalo
  22   -78.35599   NY_Giants
  23   -93.61696   Minnesota
  24   -100.14120   Arizona
  25   -108.96847   Miami
  26   -109.55536   Detroit
  27   -118.90135   Washington
  28   -121.47443   Seattle
  29   -123.09568   Jacksonville
  30   -130.80626   Las_Vegas
  31   -171.81573   LA_Chargers
  32   -174.58593   Tennes

### Calculate predictability of the method

In [20]:
numberCorrectPredictions = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]
    
    if team1Score > team2Score and eloRatings[team1ID] > eloRatings[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and eloRatings[team2ID] > eloRatings[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        numberCorrectPredictions += 1

print(f'Predictability: {numberCorrectPredictions/numGames*100:.2f}%') 


Predictability: 72.27%


In [9]:
#divisions
NFCdivisions = {'north': ["Chicago", "Detroit","Green_Bay","Minnesota"], 'south': ["Atlanta", "Carolina", "New_Orleans", "Tampa_Bay"], 'east': ["Dallas", "NY_Giants", "Philadelphia", "Washington"], 'west': ["Arizona", "LA_Rams", "San_Francisco", "Seattle"]}
AFCdivisions = {'north': ["Baltimore", "Cincinnati", "Cleveland", "Pittsburgh"], 'south': ["Houston", "Indianapolis", "Jacksonville", "Tennessee"], 'east': ["Buffalo", "Miami", "New_England", "NY_Jets", ], 'west': ["Denver", "Kansas_City", "Las_Vegas", "LA_Chargers"]}
nfc = pd.DataFrame(data=NFCdivisions)
afc = pd.DataFrame(data=AFCdivisions)
#NFCdivisions + AFCdivisions
#standings
wins = np.zeros(numTeams)
losses = np.zeros(numTeams)
ties = np.zeros(numTeams)
winsties = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        wins[team1ID] = wins[team1ID]+1
        winsties[team1ID] = winsties[team1ID]+1
        losses[team2ID] = losses[team2ID]+1
    elif team1Score < team2Score:
        wins[team2ID] = wins[team2ID]+1
        winsties[team2ID] = winsties[team2ID]+1
        losses[team1ID] = losses[team1ID]+1
    else:  
        winsties[team1ID] = winsties[team1ID]+.5
        ties[team1ID] = ties[team1ID]+1
        winsties[team2ID] = winsties[team2ID]+.5
        ties[team2ID] = ties[team2ID]+1
#df = pd.DataFrame(wins, columns = ['teamID','teams','wins','losses','ties'])
winsdf = pd.DataFrame(wins, columns = ['wins'])
lossesdf = pd.DataFrame(losses, columns = ['losses'])
tiesdf = pd.DataFrame(ties, columns = ['ties'])
winstiesdf = pd.DataFrame(winsties, columns = ['winsties'])
standings = pd.concat([teamNames, winsdf, lossesdf, tiesdf, winstiesdf], axis=1)
standings.rename(columns={1: "team_name"}, inplace = True)
standings["conference"] = ""
namesList = standings["team_name"].to_list()
newNames = []
for i in namesList:
    hi = i.strip()
    newNames.append(hi)
#print(newNames)
standings["team_name"] = newNames
conferencelist = []
divisionlist = []
for i in newNames:
    for conference, team in NFCdivisions.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        #print(conference)
        #print(team)
        if i in team:
            divisionlist.append("nfc")
            conferencelist.append(conference)
    for conference, team in AFCdivisions.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        if i in team:
            conferencelist.append(conference)
            divisionlist.append("afc")
standings["division"] = conferencelist 
standings["conference"] = divisionlist  
standings


#for i in range(32):
    #name = standings["team_name"].values[i]
    #if nfc['north'].str.contains(name).any():
        #print("hi")
    #for (columnName, columnData) in nfc.iteritems():
        #for j in range(4):
            #curTeamName = columnData.values[j]
            #if standings["team_name"].values[i].str.contains(curTeamName):
                #standings.at[i, "confrence"] = "nfc"
            #else:
        #standings["confrence"].iloc[i] = "afc"
                #standings.at[i, "confrence"] = "afc"
#standings
#test = standings[~standings["team_name"].isin(['Houston'])]
#test = standings[~standings["team_name"].str.contains("Houston")]
#test = standings[(standings["team_name"].__eq__('Houston'))]
#print(test)

,0,team_name,wins,losses,ties,winsties,conference,division
0,1,Arizona,8.0,8.0,0.0,8.0,nfc,west
1,2,Atlanta,4.0,12.0,0.0,4.0,nfc,south
2,3,Baltimore,11.0,5.0,0.0,11.0,afc,north
3,4,Buffalo,13.0,3.0,0.0,13.0,afc,east
4,5,Carolina,5.0,11.0,0.0,5.0,nfc,south
5,6,Chicago,8.0,8.0,0.0,8.0,nfc,north
6,7,Cincinnati,4.0,11.0,1.0,4.5,afc,north
7,8,Cleveland,11.0,5.0,0.0,11.0,afc,north
8,9,Dallas,6.0,10.0,0.0,6.0,nfc,east
9,10,Denver,5.0,11.0,0.0,5.0,afc,west


In [13]:
#playoff bracket
nfcplayoffs = pd.DataFrame(data = {'team_name': []})
afcplayoffs = pd.DataFrame(data = {'team_name': []})
nfcTempStandings = standings[(standings["conference"].str.contains("nfc"))]
afcTempStandings = standings[(standings["conference"].str.contains("afc"))]
#print(tempStandings)

for (columnName, columnData) in nfc.iteritems():
    playoffTeam = ""
    playoffTeamWinTies = 0
    for i in range(4):
        curTeamName = columnData.values[i]
        teamInfo = standings[(standings["team_name"].str.contains(curTeamName))]
        if teamInfo['winsties'].iloc[0] > playoffTeamWinTies:
            playoffTeamWinTies = teamInfo['winsties'].iloc[0]
            playoffTeam = teamInfo["team_name"].iloc[0]
    nfcTempStandings = nfcTempStandings[~nfcTempStandings["team_name"].str.contains(playoffTeam)]
    tempdf = pd.DataFrame(data = {'team_name': [playoffTeam]})
    nfcplayoffs = pd.concat([nfcplayoffs, tempdf])
    
nfcTempStandings = nfcTempStandings.sort_values(by='winsties', ascending=False) 
nfcwildcard = pd.DataFrame(data = {'team_name': [nfcTempStandings["team_name"].values[0], nfcTempStandings["team_name"].values[1]]})
nfcplayoffs = pd.concat([nfcplayoffs, nfcwildcard])
nfcplayoffs= nfcplayoffs.merge(standings, on='team_name', how='left')
print(nfcplayoffs)

for (columnName, columnData) in afc.iteritems():
    playoffTeam = ""
    playoffTeamWinTies = 0
    for i in range(4):
        curTeamName = columnData.values[i]
        teamInfo = standings[(standings["team_name"].str.contains(curTeamName))]
        if teamInfo['winsties'].iloc[0] > playoffTeamWinTies:
            playoffTeamWinTies = teamInfo['winsties'].iloc[0]
            playoffTeam = teamInfo["team_name"].iloc[0]
    afcTempStandings = afcTempStandings[~afcTempStandings["team_name"].str.contains(playoffTeam)]
    tempdf = pd.DataFrame(data = {'team_name': [playoffTeam]})
    afcplayoffs = pd.concat([afcplayoffs, tempdf])
    
afcTempStandings = afcTempStandings.sort_values(by='winsties', ascending=False) 
afcwildcard = pd.DataFrame(data = {'team_name': [afcTempStandings["team_name"].values[0], afcTempStandings["team_name"].values[1]]})
afcplayoffs = pd.concat([afcplayoffs, afcwildcard])
afcplayoffs= afcplayoffs.merge(standings, on='team_name', how='left')
print(afcplayoffs)

     team_name   0  wins  losses  ties  winsties conference division
0    Green_Bay  12  13.0     3.0   0.0      13.0        nfc    north
1  New_Orleans  23  12.0     4.0   0.0      12.0        nfc    south
2   Washington  32   7.0     9.0   0.0       7.0        nfc     east
3      Seattle  29  12.0     4.0   0.0      12.0        nfc     west
4    Tampa_Bay  30  11.0     5.0   0.0      11.0        nfc    south
5      LA_Rams  18  10.0     6.0   0.0      10.0        nfc     west
      team_name   0  wins  losses  ties  winsties conference division
0    Pittsburgh  27  12.0     4.0   0.0      12.0        afc    north
1  Indianapolis  14  11.0     5.0   0.0      11.0        afc    south
2       Buffalo   4  13.0     3.0   0.0      13.0        afc     east
3   Kansas_City  16  14.0     2.0   0.0      14.0        afc     west
4     Baltimore   3  11.0     5.0   0.0      11.0        afc    north
5     Cleveland   8  11.0     5.0   0.0      11.0        afc    north
